In [1]:
#Import Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split

In [16]:
#read in data
month1 = pd.read_csv('~/Documents/KU Leuven/Second semester/Advanced Analytics/ADA Assignments/ADA GitHub/ADA/assignment_1/data/test_month_1.csv')
month2 = pd.read_csv('~/Documents/KU Leuven/Second semester/Advanced Analytics/ADA Assignments/ADA GitHub/ADA/assignment_1/data/test_month_2.csv')
month3 = pd.read_csv('~/Documents/KU Leuven/Second semester/Advanced Analytics/ADA Assignments/ADA GitHub/ADA/assignment_1/data/test_month_3.csv')


Between the first and second month data, we try to figure out if certain columns changes. We then store the booleans (True/False) in a python list to indicate which features change and which remain constant. Note that we set client_id to True, so we can merge the final datasets together using this column

In [17]:
notequal = []
for col in list(month1):
    notequal.append(not month1[col].equals(month2[col]))
notequal[0] = True
print(notequal)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, True, False, True, True]


Repeat for the first and third month. Note that we  append an extra True value at the end, because the third month dataset has an additional column (Target), that was previously not available in the other two datasets.

In [18]:
notequal2 = []
for col in list(month1):
    notequal2.append(not month1[col].equals(month3[col]))
notequal2[0] = True
print(notequal2)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, True, False, True, True]


In [24]:
month1.columns

Index(['client_id', 'homebanking_active', 'has_homebanking',
       'has_insurance_21', 'has_insurance_23', 'has_life_insurance_fixed_cap',
       'has_life_insurance_decreasing_cap', 'has_fire_car_other_insurance',
       'has_personal_loan', 'has_mortgage_loan', 'has_current_account',
       'has_pension_saving', 'has_savings_account',
       'has_savings_account_starter', 'has_current_account_starter',
       'bal_insurance_21', 'bal_insurance_23', 'cap_life_insurance_fixed_cap',
       'cap_life_insurance_decreasing_cap', 'prem_fire_car_other_insurance',
       'bal_personal_loan', 'bal_mortgage_loan', 'bal_current_account',
       'bal_pension_saving', 'bal_savings_account',
       'bal_savings_account_starter', 'bal_current_account_starter',
       'visits_distinct_so', 'visits_distinct_so_areas', 'customer_since_all',
       'customer_since_bank', 'customer_gender', 'customer_birth_date',
       'customer_postal_code', 'customer_occupation_code',
       'customer_self_employed',

In [19]:
#rename the columns in month 3, to prevent features from replacing each other when merging
month3 = month3.add_suffix("_3")
month3 = month3.rename(columns={"client_id_3":"client_id"})
month3.head()

,client_id,homebanking_active_3,has_homebanking_3,has_insurance_21_3,has_insurance_23_3,has_life_insurance_fixed_cap_3,has_life_insurance_decreasing_cap_3,has_fire_car_other_insurance_3,has_personal_loan_3,has_mortgage_loan_3,...,customer_since_all_3,customer_since_bank_3,customer_gender_3,customer_birth_date_3,customer_postal_code_3,customer_occupation_code_3,customer_self_employed_3,customer_education_3,customer_children_3,customer_relationship_3
0,ccf4cd93d5c32cd8a59809d54b4d53ac,0,0,0,0,0,0,0,0,0,...,1981-01,1981-01,2,1937-01,9860,9.0,0,5.0,NaN,single
1,56605a660d18549592653ff6941186f1,0,0,0,0,0,0,1,0,0,...,1993-03,1993-03,2,1941-04,2491,9.0,0,NaN,NaN,single
2,bda5f84c05e5695a7ec10550b457890f,0,0,0,0,0,1,1,0,1,...,1993-08,2005-06,1,1969-04,1770,9.0,0,3.0,mature,couple
3,a2f1c04bc3acf2222e658a897400798f,0,0,0,0,0,0,1,0,0,...,1978-08,2009-01,1,1952-09,7750,9.0,0,NaN,no,couple
4,e83aadc3b0d25dbc12a35551afa25807,0,0,0,0,0,0,0,0,0,...,2014-04,2014-04,2,1962-08,2270,9.0,1,NaN,grownup,couple


Next, we merge the three datasets, only joining features that change from month to months. Anything constant such as date of birth only appear once in the final dataset. All features that appear repeatedly from month to month will have a suffix of a number indicating the feature for the particular month. For example, "has_homebanking_1" is the associated feature that indicates whether a client has homebanking for month 1, while "has_homebanking_2" is for month 2, etc.

In [53]:
full_df = pd.merge(month1, month2.loc[:,notequal], left_on="client_id", right_on="client_id", how="inner", suffixes=["_1", "_2"])
full_df.head()

,client_id,homebanking_active_1,has_homebanking_1,has_insurance_21_1,has_insurance_23_1,has_life_insurance_fixed_cap_1,has_life_insurance_decreasing_cap_1,has_fire_car_other_insurance_1,has_personal_loan_1,has_mortgage_loan_1,...,bal_current_account_2,bal_pension_saving_2,bal_savings_account_2,bal_savings_account_starter_2,bal_current_account_starter_2,visits_distinct_so_2,visits_distinct_so_areas_2,customer_self_employed_2,customer_children_2,customer_relationship_2
0,ccf4cd93d5c32cd8a59809d54b4d53ac,0,0,0,0,0,0,0,0,0,...,1260,0,5190,0,0,1.0,1.0,0,NaN,single
1,56605a660d18549592653ff6941186f1,0,0,0,0,0,0,1,0,0,...,30,0,15010,0,0,1.0,1.0,0,NaN,single
2,bda5f84c05e5695a7ec10550b457890f,0,0,0,0,0,1,1,0,1,...,3290,0,3700,0,0,1.0,1.0,0,mature,couple
3,a2f1c04bc3acf2222e658a897400798f,0,0,0,0,0,0,1,0,0,...,3520,0,27500,0,0,1.0,1.0,0,no,couple
4,e83aadc3b0d25dbc12a35551afa25807,0,0,0,0,0,0,0,0,0,...,0,0,10190,0,0,1.0,1.0,1,grownup,couple


In [54]:
full_df = pd.merge(full_df, month3.loc[:,notequal2], left_on="client_id", right_on="client_id", how="inner")
full_df.head()

,client_id,homebanking_active_1,has_homebanking_1,has_insurance_21_1,has_insurance_23_1,has_life_insurance_fixed_cap_1,has_life_insurance_decreasing_cap_1,has_fire_car_other_insurance_1,has_personal_loan_1,has_mortgage_loan_1,...,bal_current_account_3,bal_pension_saving_3,bal_savings_account_3,bal_savings_account_starter_3,bal_current_account_starter_3,visits_distinct_so_3,visits_distinct_so_areas_3,customer_self_employed_3,customer_children_3,customer_relationship_3
0,ccf4cd93d5c32cd8a59809d54b4d53ac,0,0,0,0,0,0,0,0,0,...,1590,0,5190,0,0,1.0,1.0,0,NaN,single
1,56605a660d18549592653ff6941186f1,0,0,0,0,0,0,1,0,0,...,0,0,15010,0,0,1.0,1.0,0,NaN,single
2,bda5f84c05e5695a7ec10550b457890f,0,0,0,0,0,1,1,0,1,...,4200,0,3700,0,0,1.0,1.0,0,mature,couple
3,a2f1c04bc3acf2222e658a897400798f,0,0,0,0,0,0,1,0,0,...,3960,0,27500,0,0,1.0,1.0,0,no,couple
4,e83aadc3b0d25dbc12a35551afa25807,0,0,0,0,0,0,0,0,0,...,0,0,10190,0,0,1.0,1.0,1,grownup,couple


To avoid having multiple repeated features for different months, we combine them either in sequences of binary or an aggregate of count.

In [55]:
#these features have a binary value indicating presence/absence
status_features = list(month1.loc[:, notequal].columns)[:14]
status_features

['client_id',
 'homebanking_active',
 'has_homebanking',
 'has_insurance_21',
 'has_insurance_23',
 'has_life_insurance_fixed_cap',
 'has_life_insurance_decreasing_cap',
 'has_fire_car_other_insurance',
 'has_personal_loan',
 'has_mortgage_loan',
 'has_current_account',
 'has_pension_saving',
 'has_savings_account',
 'has_savings_account_starter']

For the first 13 features (excluding client_id), we can concat the binary features into a string sequence to indicate a 3 month combination pattern:

In [56]:
for col in status_features:
    if col != "client_id":
        
        full_df[str(col) + "_seq"] = full_df.filter(regex="^"+str(col)+"_[0-9]").apply(lambda x: ''.join(x.astype(str)), axis=1)

In [57]:
full_df.filter(regex=".*_seq").head()

,homebanking_active_seq,has_homebanking_seq,has_insurance_21_seq,has_insurance_23_seq,has_life_insurance_fixed_cap_seq,has_life_insurance_decreasing_cap_seq,has_fire_car_other_insurance_seq,has_personal_loan_seq,has_mortgage_loan_seq,has_current_account_seq,has_pension_saving_seq,has_savings_account_seq,has_savings_account_starter_seq
0,000,000,000,000,000,000,000,000,000,111,000,111,000
1,000,000,000,000,000,000,111,000,000,111,000,111,000
2,000,000,000,000,000,111,111,000,111,111,000,111,000
3,000,000,000,000,000,000,111,000,000,111,000,111,000
4,000,000,000,000,000,000,000,000,000,000,000,111,000


We can also simply aggregate the binary values, to count how many positive occurances a particular client had in the 3 month span for these 14 features:

In [58]:
for col in status_features:
    if col != "client_id":
        
        full_df[str(col) + "_count"] = full_df.filter(regex="^"+str(col)+"_[0-9]").sum(axis=1)

In [59]:
full_df.head()

,client_id,homebanking_active_1,has_homebanking_1,has_insurance_21_1,has_insurance_23_1,has_life_insurance_fixed_cap_1,has_life_insurance_decreasing_cap_1,has_fire_car_other_insurance_1,has_personal_loan_1,has_mortgage_loan_1,...,has_insurance_23_count,has_life_insurance_fixed_cap_count,has_life_insurance_decreasing_cap_count,has_fire_car_other_insurance_count,has_personal_loan_count,has_mortgage_loan_count,has_current_account_count,has_pension_saving_count,has_savings_account_count,has_savings_account_starter_count
0,ccf4cd93d5c32cd8a59809d54b4d53ac,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,3,0
1,56605a660d18549592653ff6941186f1,0,0,0,0,0,0,1,0,0,...,0,0,0,3,0,0,3,0,3,0
2,bda5f84c05e5695a7ec10550b457890f,0,0,0,0,0,1,1,0,1,...,0,0,3,3,0,3,3,0,3,0
3,a2f1c04bc3acf2222e658a897400798f,0,0,0,0,0,0,1,0,0,...,0,0,0,3,0,0,3,0,3,0
4,e83aadc3b0d25dbc12a35551afa25807,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,0


For Balance features, we take the difference between the third and first month, to track the change over time. Note that this also includes visits

In [60]:
balance_features = list(month1.loc[:, notequal].columns)[14:28]
balance_features

['has_current_account_starter',
 'bal_insurance_21',
 'bal_insurance_23',
 'cap_life_insurance_fixed_cap',
 'cap_life_insurance_decreasing_cap',
 'prem_fire_car_other_insurance',
 'bal_personal_loan',
 'bal_mortgage_loan',
 'bal_current_account',
 'bal_pension_saving',
 'bal_savings_account',
 'bal_savings_account_starter',
 'bal_current_account_starter',
 'visits_distinct_so']

In [61]:
for col in balance_features:
    full_df[str(col)+"_diff"] = np.array(full_df.filter(regex="^"+str(col)+"_3")) - np.array(full_df.filter(regex="^"+str(col)+"_1"))

Convert Column into datetime object

In [62]:
full_df['customer_since_all_d']=pd.to_datetime(full_df['customer_since_all'], format='%Y-%m-%d')
full_df['customer_birth_date_d']=pd.to_datetime(full_df['customer_birth_date'], format='%Y-%m-%d')


Compute the age of the client

In [85]:
full_df['mob']=(pd.to_datetime('2018-10-01')-full_df['customer_since_all_d'])/np.timedelta64(1, 'M')
full_df['mob'].describe()

count    27189.000000
mean       230.553098
std        138.196507
min          8.969383
25%        118.967535
50%        228.998542
75%        315.998275
max        764.993121
Name: mob, dtype: float64

In [64]:
full_df['age']=2018-pd.DatetimeIndex(full_df['customer_birth_date_d']).year
full_df['mob']=np.where(full_df['customer_since_all'].notna(),(pd.to_datetime('2018-10-01')-full_df['customer_since_all_d'])/np.timedelta64(1, 'M').astype('int'),None)


Instead of looking at each feature's sequence or count separately e.g `has_homebanking_seq`, we make a column `insurance` is a higher level feature that is 1 if either has_insurance_21, has_insurance_23, has_life_insurance_fixed_cap, etc. is 1 otherwise it is 0. We do this for all three months and at the end we can have `insurance_seq`, which is the 3 month sequence of the higher level variable.

In [65]:
list(full_df)

['client_id',
 'homebanking_active_1',
 'has_homebanking_1',
 'has_insurance_21_1',
 'has_insurance_23_1',
 'has_life_insurance_fixed_cap_1',
 'has_life_insurance_decreasing_cap_1',
 'has_fire_car_other_insurance_1',
 'has_personal_loan_1',
 'has_mortgage_loan_1',
 'has_current_account_1',
 'has_pension_saving_1',
 'has_savings_account_1',
 'has_savings_account_starter_1',
 'has_current_account_starter_1',
 'bal_insurance_21_1',
 'bal_insurance_23_1',
 'cap_life_insurance_fixed_cap_1',
 'cap_life_insurance_decreasing_cap_1',
 'prem_fire_car_other_insurance_1',
 'bal_personal_loan_1',
 'bal_mortgage_loan_1',
 'bal_current_account_1',
 'bal_pension_saving_1',
 'bal_savings_account_1',
 'bal_savings_account_starter_1',
 'bal_current_account_starter_1',
 'visits_distinct_so_1',
 'visits_distinct_so_areas_1',
 'customer_since_all',
 'customer_since_bank',
 'customer_gender',
 'customer_birth_date',
 'customer_postal_code',
 'customer_occupation_code',
 'customer_self_employed_1',
 'customer

In [66]:
full_df['insurance_3']=full_df[['has_insurance_21_3','has_insurance_23_3','has_life_insurance_fixed_cap_3','has_life_insurance_decreasing_cap_3','has_fire_car_other_insurance_3']].apply(lambda row: row.max(),axis=1)
full_df['insurance_2']=full_df[['has_insurance_21_2','has_insurance_23_2','has_life_insurance_fixed_cap_2','has_life_insurance_decreasing_cap_2','has_fire_car_other_insurance_2']].apply(lambda row: row.max(),axis=1)
full_df['insurance_1']=full_df[['has_insurance_21_1','has_insurance_23_1','has_life_insurance_fixed_cap_1','has_life_insurance_decreasing_cap_1','has_fire_car_other_insurance_1']].apply(lambda row: row.max(),axis=1)
full_df['savings_3']=full_df[['has_pension_saving_3','has_savings_account_3','has_savings_account_starter_3','has_current_account_starter_3']].apply(lambda row: row.max(),axis=1)
#note has_savings_account_starter did not change from month 1 to month 2 for all clients
full_df['savings_2']=full_df[['has_pension_saving_2','has_savings_account_2','has_savings_account_starter_2','has_current_account_starter_2']].apply(lambda row: row.max(),axis=1)
full_df['savings_1']=full_df[['has_pension_saving_1','has_savings_account_1','has_savings_account_starter_1','has_current_account_starter_1']].apply(lambda row: row.max(),axis=1)
full_df['loan_3']=full_df[['has_personal_loan_3','has_mortgage_loan_3','has_current_account_3']].apply(lambda row: row.max(),axis=1)
full_df['loan_2']=full_df[['has_personal_loan_2','has_mortgage_loan_2','has_current_account_2']].apply(lambda row: row.max(),axis=1)
full_df['loan_1']=full_df[['has_personal_loan_1','has_mortgage_loan_1','has_current_account_1']].apply(lambda row: row.max(),axis=1)

#checks whether the customer has an account with the bank for a particular month
full_df['has_account_3']=full_df[['insurance_3','savings_3','loan_3']].apply(lambda row: row.max(),axis=1)
full_df['has_account_2']=full_df[['insurance_2','savings_2','loan_2']].apply(lambda row: row.max(),axis=1)
full_df['has_account_1']=full_df[['insurance_1','savings_1','loan_1']].apply(lambda row: row.max(),axis=1)


#Total balance for a particular month
full_df['bal_insurance_3']=full_df[['bal_insurance_21_3','bal_insurance_23_3']].apply(lambda row: row.sum(),axis=1)
full_df['bal_insurance_2']=full_df[['bal_insurance_21_2','bal_insurance_23_2']].apply(lambda row: row.sum(),axis=1)
full_df['bal_insurance_1']=full_df[['bal_insurance_21_1','bal_insurance_23_1']].apply(lambda row: row.sum(),axis=1)

full_df['bal_savings_3']=full_df[['bal_pension_saving_3','bal_savings_account_3','bal_savings_account_starter_3','bal_current_account_starter_3']].apply(lambda row: row.sum(),axis=1)
full_df['bal_savings_2']=full_df[['bal_pension_saving_2','bal_savings_account_2','bal_savings_account_starter_2','bal_current_account_starter_2']].apply(lambda row: row.sum(),axis=1)
full_df['bal_savings_1']=full_df[['bal_pension_saving_1','bal_savings_account_1','bal_savings_account_starter_1','bal_current_account_starter_1']].apply(lambda row: row.sum(),axis=1)

full_df['bal_loan_3']=full_df[['bal_personal_loan_3','bal_mortgage_loan_3','bal_current_account_3']].apply(lambda row: row.sum(),axis=1)
full_df['bal_loan_2']=full_df[['bal_personal_loan_2','bal_mortgage_loan_2','bal_current_account_2']].apply(lambda row: row.sum(),axis=1)
full_df['bal_loan_1']=full_df[['bal_personal_loan_1','bal_mortgage_loan_1','bal_current_account_1']].apply(lambda row: row.sum(),axis=1)

#debt ratio for each month
full_df['debt_ratio_3']=np.where(full_df['bal_loan_3']/full_df['bal_savings_3']>=0,full_df['bal_loan_3']/full_df['bal_savings_3'],0)
full_df['debt_ratio_2']=np.where(full_df['bal_loan_2']/full_df['bal_savings_2']>=0,full_df['bal_loan_2']/full_df['bal_savings_2'],0)
full_df['debt_ratio_1']=np.where(full_df['bal_loan_1']/full_df['bal_savings_1']>=0,full_df['bal_loan_1']/full_df['bal_savings_1'],0)

#gender
full_df['gender'] = np.where(full_df['customer_gender']==1, 1, 0)

In [67]:
#sumarize the binary variables into sequences rather than keeping individual columns

sequence_var = ['insurance', 'savings', 'loan', 'has_account']

for col in sequence_var:
    full_df[str(col)+"_seq"] = full_df.filter(regex="^" +str(col) +"_[0-9]").apply(lambda x: ''.join(x.astype(str)), axis=1)

#count gives the number of months the indicator was present in a 3-month span
for col in sequence_var:
    full_df[str(col)+"_count"] = full_df.filter(regex="^" +str(col) +"_[0-9]").sum(axis=1)


In [68]:
#for balances, we take the difference between the third month and the first month

balance_var = ['bal_insurance', 'bal_savings', 'bal_loan']

for col in balance_var:
    full_df[str(col)+"_diff"] = np.array(full_df.filter(regex="^" + str(col)+"_3")) - np.array(full_df.filter(regex="^" + str(col)+"_1"))

In [69]:
list(full_df)

['client_id',
 'homebanking_active_1',
 'has_homebanking_1',
 'has_insurance_21_1',
 'has_insurance_23_1',
 'has_life_insurance_fixed_cap_1',
 'has_life_insurance_decreasing_cap_1',
 'has_fire_car_other_insurance_1',
 'has_personal_loan_1',
 'has_mortgage_loan_1',
 'has_current_account_1',
 'has_pension_saving_1',
 'has_savings_account_1',
 'has_savings_account_starter_1',
 'has_current_account_starter_1',
 'bal_insurance_21_1',
 'bal_insurance_23_1',
 'cap_life_insurance_fixed_cap_1',
 'cap_life_insurance_decreasing_cap_1',
 'prem_fire_car_other_insurance_1',
 'bal_personal_loan_1',
 'bal_mortgage_loan_1',
 'bal_current_account_1',
 'bal_pension_saving_1',
 'bal_savings_account_1',
 'bal_savings_account_starter_1',
 'bal_current_account_starter_1',
 'visits_distinct_so_1',
 'visits_distinct_so_areas_1',
 'customer_since_all',
 'customer_since_bank',
 'customer_gender',
 'customer_birth_date',
 'customer_postal_code',
 'customer_occupation_code',
 'customer_self_employed_1',
 'customer

In [70]:
#customer self employed status summarized

full_df['customer_self_employed_seq'] = full_df.filter(regex="^customer_self_employed_[0-9]").apply(lambda x: ''.join(x.astype(str)), axis=1)

full_df['customer_self_employed_count'] = full_df.filter(regex="^customer_self_employed_[0-9]").sum(axis=1)





#customer relationship

# Outliers and missing values


### Debt ratio
For customers without savings balance, the debt ratio is defined as infinite therefore we set a maximum cap for these cases

In [72]:
full_df[(full_df['bal_savings_1']==0)&(full_df['bal_loan_1']>0)]['debt_ratio_1'].count() # Check for de ratios as infinity -1775 obs
full_df[(full_df['bal_savings_2']==0)&(full_df['bal_loan_2']>0)]['debt_ratio_2'].count() # Check for de ratios as infinity - 1769 obs
full_df[(full_df['bal_savings_3']==0)&(full_df['bal_loan_3']>0)]['debt_ratio_3'].count() # Check for de ratios as infinity - 1763 obs
full_df[full_df['debt_ratio_1'] >1][{'debt_ratio_1','bal_savings_1','bal_loan_1'}]

,bal_savings_1,debt_ratio_1,bal_loan_1
2,3700,42.667568,157870
5,0,inf,8970
20,17990,5.024458,90390
30,13000,2.059231,26770
50,480,21.562500,10350
...,...,...,...
27257,0,inf,10390
27262,18800,2.307447,43380
27274,6930,19.093795,132320
27290,15560,8.059126,125400


In [73]:
full_df.loc[(full_df['bal_savings_1']==0)&(full_df['bal_loan_1']>0), 'debt_ratio_1'] = 100
full_df.loc[full_df['debt_ratio_1']>100, 'debt_ratio_1'] = 100
full_df.loc[(full_df['bal_savings_2']==0)&(full_df['bal_loan_2']>0), 'debt_ratio_2'] = 100
full_df.loc[full_df['debt_ratio_2']>100, 'debt_ratio_2'] = 100
full_df.loc[(full_df['bal_savings_3']==0)&(full_df['bal_loan_3']>0), 'debt_ratio_3'] = 100
full_df.loc[full_df['debt_ratio_3']>100, 'debt_ratio_3'] = 100

In [74]:
full_df['debt_ratio_diff']=full_df['debt_ratio_3']-full_df['debt_ratio_1']
full_df['debt_ratio_diff'].describe()

count    27300.000000
mean        -0.046978
std          2.744637
min       -100.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         98.562057
Name: debt_ratio_diff, dtype: float64

### Age

According to the data there are customers older than 100 years old, we will set a cap for these cases instead of deleting these observations


In [75]:
full_df['age'].describe()

count    27300.000000
mean        56.420952
std         19.996154
min         18.000000
25%         41.000000
50%         58.000000
75%         73.000000
max        139.000000
Name: age, dtype: float64

In [76]:
full_df[full_df['age']>100] # Check for ages above 100 - 129 obs
full_df.loc[full_df['age']>100, 'age'] = 100

### MoB -Months on Books

The number of months the person has been a customer for the bank. For 234 missing values, the average of the variable is assigned 


In [86]:
full_df['mob'].describe()

count    27189.000000
mean       230.553098
std        138.196507
min          8.969383
25%        118.967535
50%        228.998542
75%        315.998275
max        764.993121
Name: mob, dtype: float64

In [87]:
full_df[full_df['mob'].isna()==True][{'mob','customer_since_all','customer_since_bank','customer_birth_date','age'}] #111 obs without customer since all

,customer_birth_date,age,customer_since_all,mob,customer_since_bank
293,1968-11,50,NaN,NaN,NaN
474,1964-11,54,NaN,NaN,NaN
604,1971-06,47,NaN,NaN,NaN
619,1962-04,56,NaN,NaN,NaN
634,1961-10,57,NaN,NaN,NaN
...,...,...,...,...,...
26517,1939-04,79,NaN,NaN,NaN
26981,1963-10,55,NaN,NaN,NaN
27037,1946-05,72,NaN,NaN,NaN
27142,1949-02,69,NaN,NaN,NaN


In [89]:
full_df['mob'].mean()


230.5530980514049

In [90]:
full_df.loc[full_df['mob'].isna(), 'mob'] = full_df['mob'].mean()

In [92]:
full_df[full_df['mob'].isna()==True]

,client_id,homebanking_active_1,has_homebanking_1,has_insurance_21_1,has_insurance_23_1,has_life_insurance_fixed_cap_1,has_life_insurance_decreasing_cap_1,has_fire_car_other_insurance_1,has_personal_loan_1,has_mortgage_loan_1,...,insurance_count,savings_count,loan_count,has_account_count,bal_insurance_diff,bal_savings_diff,bal_loan_diff,customer_self_employed_seq,customer_self_employed_count,debt_ratio_diff


In [93]:
full_df['mob'].describe()

count    27300.000000
mean       230.553098
std        137.915262
min          8.969383
25%        118.967535
50%        230.017043
75%        315.998275
max        764.993121
Name: mob, dtype: float64

### Customer Children

The current possible values are: NA, yes, no, adolescent, young, baby, mature, etc. We will remap these values to 0, 1, 2. Here 0 indicates missing, 1 indicates no and 2 indicates a children present (regardless of age). We will then proceed with creating sequence variable 'customer_children_seq' and a categorical variable with the maximum value of the 3 months 'customer_children_cat'


In [94]:
def clean_children(text):
    if pd.isnull(text):
        new_val = 0
    elif str(text) == "no":
        new_val = 1
    else:
        new_val = 2
    
    return new_val

full_df["customer_children_clean_1"]  = full_df['customer_children_1'].apply(clean_children)
full_df["customer_children_clean_2"]  = full_df['customer_children_2'].apply(clean_children)
full_df["customer_children_clean_3"]  = full_df['customer_children_3'].apply(clean_children)

full_df['customer_children_seq'] = full_df.filter(regex="^customer_children_clean_[0-9]").apply(lambda x: ''.join(x.astype(str)), axis=1)
full_df['customer_children_cat'] = full_df.filter(regex="^customer_children_clean_[0-9]").apply(lambda x:x.max(), axis=1)

full_df['customer_children_seq'].value_counts()
full_df['customer_children_cat'].value_counts()


1    9763
0    9634
2    7903
Name: customer_children_cat, dtype: int64

### Customer Relationship

Customer relationship can take the following values: NA (=0), single (=1), couple (=2). We will remap it as 0, 1, 2 respectively and compute the sequence of the relationship over a 3 month period and the categorical variable defined as the maximum over the 3 months

In [95]:
def clean_relationship(status):
    if pd.isnull(status):
        new_val = 0
    elif status == "single":
        new_val = 1
    else:
        new_val = 2

    return new_val

full_df["customer_relationship_clean_1"]  = full_df['customer_relationship_1'].apply(clean_children)
full_df["customer_relationship_clean_2"]  = full_df['customer_relationship_2'].apply(clean_children)
full_df["customer_relationship_clean_3"]  = full_df['customer_relationship_3'].apply(clean_children)

full_df['customer_relationship_seq'] = full_df.filter(regex="^customer_relationship_clean_[0-9]").apply(lambda x: ''.join(x.astype(str)), axis=1)
full_df['customer_relationship_cat'] = full_df.filter(regex="^customer_relationship_clean_[0-9]").apply(lambda x:x.max(), axis=1)

full_df['customer_relationship_seq'].value_counts()
full_df['customer_relationship_cat'].value_counts()

2    21287
0     6013
Name: customer_relationship_cat, dtype: int64

In [96]:
full_df.head()

,client_id,homebanking_active_1,has_homebanking_1,has_insurance_21_1,has_insurance_23_1,has_life_insurance_fixed_cap_1,has_life_insurance_decreasing_cap_1,has_fire_car_other_insurance_1,has_personal_loan_1,has_mortgage_loan_1,...,customer_children_clean_1,customer_children_clean_2,customer_children_clean_3,customer_children_seq,customer_children_cat,customer_relationship_clean_1,customer_relationship_clean_2,customer_relationship_clean_3,customer_relationship_seq,customer_relationship_cat
0,ccf4cd93d5c32cd8a59809d54b4d53ac,0,0,0,0,0,0,0,0,0,...,0,0,0,000,0,2,2,2,222,2
1,56605a660d18549592653ff6941186f1,0,0,0,0,0,0,1,0,0,...,0,0,0,000,0,2,2,2,222,2
2,bda5f84c05e5695a7ec10550b457890f,0,0,0,0,0,1,1,0,1,...,2,2,2,222,2,2,2,2,222,2
3,a2f1c04bc3acf2222e658a897400798f,0,0,0,0,0,0,1,0,0,...,1,1,1,111,1,2,2,2,222,2
4,e83aadc3b0d25dbc12a35551afa25807,0,0,0,0,0,0,0,0,0,...,2,2,2,222,2,2,2,2,222,2


# Descriptive statistics per group

### Differences M1 and M3


In [97]:
list(full_df.filter(regex="^(.*_[0-9]$)"))

['homebanking_active_1',
 'has_homebanking_1',
 'has_insurance_21_1',
 'has_insurance_23_1',
 'has_life_insurance_fixed_cap_1',
 'has_life_insurance_decreasing_cap_1',
 'has_fire_car_other_insurance_1',
 'has_personal_loan_1',
 'has_mortgage_loan_1',
 'has_current_account_1',
 'has_pension_saving_1',
 'has_savings_account_1',
 'has_savings_account_starter_1',
 'has_current_account_starter_1',
 'bal_insurance_21_1',
 'bal_insurance_23_1',
 'cap_life_insurance_fixed_cap_1',
 'cap_life_insurance_decreasing_cap_1',
 'prem_fire_car_other_insurance_1',
 'bal_personal_loan_1',
 'bal_mortgage_loan_1',
 'bal_current_account_1',
 'bal_pension_saving_1',
 'bal_savings_account_1',
 'bal_savings_account_starter_1',
 'bal_current_account_starter_1',
 'visits_distinct_so_1',
 'visits_distinct_so_areas_1',
 'customer_self_employed_1',
 'customer_children_1',
 'customer_relationship_1',
 'homebanking_active_2',
 'has_homebanking_2',
 'has_insurance_21_2',
 'has_insurance_23_2',
 'has_life_insurance_fix

In [98]:
#exclude all features with _[0-9]

exclude_lst = list(full_df.filter(regex="^(.*_[1,2]$)"))

final_df = full_df.drop(exclude_lst, axis=1)

In [99]:
final_df.head()

,client_id,customer_since_all,customer_since_bank,customer_gender,customer_birth_date,customer_postal_code,customer_occupation_code,customer_education,homebanking_active_3,has_homebanking_3,...,bal_loan_diff,customer_self_employed_seq,customer_self_employed_count,debt_ratio_diff,customer_children_clean_3,customer_children_seq,customer_children_cat,customer_relationship_clean_3,customer_relationship_seq,customer_relationship_cat
0,ccf4cd93d5c32cd8a59809d54b4d53ac,1981-01,1981-01,2,1937-01,9860,9.0,5.0,0,0,...,790,000,0,0.182327,0,000,0,2,222,2
1,56605a660d18549592653ff6941186f1,1993-03,1993-03,2,1941-04,2491,9.0,NaN,0,0,...,-30,000,0,-0.001999,0,000,0,2,222,2
2,bda5f84c05e5695a7ec10550b457890f,1993-08,2005-06,1,1969-04,1770,9.0,3.0,0,0,...,-410,000,0,-0.110811,2,222,2,2,222,2
3,a2f1c04bc3acf2222e658a897400798f,1978-08,2009-01,1,1952-09,7750,9.0,NaN,0,0,...,920,000,0,0.033455,1,111,1,2,222,2
4,e83aadc3b0d25dbc12a35551afa25807,2014-04,2014-04,2,1962-08,2270,9.0,NaN,0,0,...,0,111,3,0.000000,2,222,2,2,222,2


In [100]:
final_df.to_csv("data_test_final.csv", index=False)

In [103]:
len(list(final_df))

112